# 2019 VU Data Mining Techniques Cup

link: https://www.kaggle.com/t/81fd4b6b248c4642930d5c1013af967a

__TASK__: "Predict what hotels properties listed as a result of a hotel search a user is most likely to click on."

 __Evaluation metric__: Normalized Discounted Cumulative Gain (NDCG).
 
 
### DATASET

For a very nice overview on the data fields of the task,  refer to https://www.kaggle.com/c/expedia-personalized-sort/data .

* Note: test.csv does not contain the following columns: position, click_bool, gross_bookings_usd, nor booking_bool


## Dicussion Board

#### Done!


# Import libraries

In [0]:
# Install dependencies
!pip install PyDrive
!pip install kaggle

In [0]:
# Import libraries
import os
import sys
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from scipy import stats
from google.colab import files

import time
import json
import math
import copy 
import random
import datetime
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from pandas.api.types import is_numeric_dtype
from sklearn.ensemble import GradientBoostingRegressor


pd.set_option('display.max_column',None)
pd.set_option('display.max_rows',None)
pd.set_option('display.max_seq_items',None)
pd.set_option('display.max_colwidth', 500)
pd.set_option('expand_frame_repr', True)

In [0]:
if "training_set_VU_DM.csv" not in os.listdir():
    !mkdir .kaggle
    # Create API Token for my account
    token = {"username":"pollendo","key":"8666602a92bda984143ce79eca66ae75"}
    with open(".kaggle/kaggle.json", "w") as file:
        json.dump(token, file)
    !kaggle config set -n path -v/content

    !cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
    !chmod 600 /root/.kaggle/kaggle.json

    # download
    !kaggle competitions download -c vu-dmt-2assignment
    # unzip
    !unzip training_set_VU_DM.csv
    !unzip test_set_VU_DM.csv
    !unzip submission_sample.csv

# Read Data

In [0]:
# Set these global variables to switch between local validation or Kaggle submission
DATA_FRACTION = 0.01
TESTING = False

if TESTING:
    DATA_FRACTION = 1.0
    with open("test_set_VU_DM.csv",'r') as f_test:
        test_rows = sum(1 for row in f_test)
    test_set = pd.read_csv('test_set_VU_DM.csv', nrows=round(DATA_FRACTION*test_rows))


with open("training_set_VU_DM.csv",'r') as f_train:
    train_rows = sum(1 for row in f_train)

training_set = pd.read_csv('training_set_VU_DM.csv', nrows=round(DATA_FRACTION*train_rows))

# Feature Generation

In [0]:
################ Normalize df per feat #################
def norm(df, feature):
    warnings.filterwarnings('ignore')

    # Create x, where x the 'scores' column's values as floats
    x = df[[feature]].values.astype(float)

    # Create a minimum and maximum processor object
    min_max_scaler = preprocessing.MinMaxScaler()

    # Create an object to transform the data to fit minmax processor
    x_scaled = min_max_scaler.fit_transform(x)

    # Run the normalizer on the dataframe
    df[feature] = x_scaled

    return df


###################### TARGET #########################
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def get_target(df):
    df['target'] = 0.8 * 0.5 * (df.booking_bool.astype(float) + df.click_bool.astype(float)) \
        + 0.2 * sigmoid(np.log(df.position.astype(float))*df.click_bool.astype(float)
                 + (1/df.position.astype(float))*(1-df.click_bool.astype(float)))
    
    df = norm(df, 'target')

    return df


####################### Time ##########################
def get_time_feat(df):
    df.insert(2, 'time', df.date_time.dt.time)
    df.insert(3, 'weekday', df.date_time.dt.dayofweek)
    df.insert(4, 'month', df.date_time.dt.month)
    df.date_time = df.date_time.dt.date
    df = df.drop(['date_time'], axis=1)
    df = df.drop(['time'], axis=1)   
    
    return df


################### Per search score ###################
def ratio(data, feature, new_feat_name, inv=True):

    d = {'srch_id': data.srch_id.values, feature: data[feature].values}
    df = pd.DataFrame(data=d)
    
    avg_feat_per_scrch = df
    avg_feat_per_scrch = avg_feat_per_scrch.rename(columns={feature: 'median_feat'})
    avg_feat_per_scrch = avg_feat_per_scrch.groupby('srch_id', as_index=False).median()
    
    df = df.merge(avg_feat_per_scrch, on='srch_id', how='left')

    if not inv:
        data[new_feat_name] = df.median_feat / (df[feature] + 1)
    else:
        data[new_feat_name] = df[feature] / ( df.median_feat + 1)

    return data


def get_hotelworth(data):
        data = ratio(data, 'price_usd', 'diff_price_per_srch', False)
        data = ratio(data, 'prop_starrating', 'diff_star1_per_srch') 
        data = ratio(data, 'prop_review_score', 'diff_star2_per_srch')
        data = ratio(data, 'prop_location_score1', 'diff_loc1_per_srch')
        data = ratio(data, 'prop_location_score2', 'diff_loc2_per_srch')

        data.diff_star1_per_srch = data.diff_star1_per_srch.fillna(0)
        data.diff_star2_per_srch = data.diff_star2_per_srch.fillna(0)
        data.diff_loc2_per_srch = data.diff_loc2_per_srch.fillna(0)
        data.diff_price_per_srch = data.diff_price_per_srch.fillna(0)

        df_hotel_worth = 0.3*(0.8*data['diff_star1_per_srch'] + 0.2*data['diff_star2_per_srch']) \
        + 0.3*(0.9*data['diff_loc2_per_srch'] + 0.1*data['diff_loc1_per_srch'])  \
        + 0.1*(np.log(data['diff_price_per_srch']))

                    
        data['hotel_worth'] = df_hotel_worth / 5
        
        # Normalize hotel_woth by scrh
#         srch_ls = df.srch_id.unique()
#         for e,srch in enumerate(srch_ls):
#           print("{} / {}".format(e, len(srch_ls)))
#           df[df.srch_id == srch] = norm(df[df.srch_id == srch], 'hotel_worth')

        return data
    

############# User's Preferences score #################
def gaussian(x, mu, sig):
    return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))

def per_cent_avg_std(df, category1, category2):
    """
    NOTE: category1 has to be the user's entry
    
    category1: str of feature
    category2: str of feature
    
    """
    # Get a df of the two categories only
    diff_df = df[[category1, category2]]
    
    # Clean NaNs
    diff_df = diff_df[diff_df[category1].notnull()]
    diff_df = diff_df[diff_df[category2].notnull()]

    # Get the diff on % scale
    diff_price = np.abs(diff_df[category1] - diff_df[category2])/diff_df[category1]

    diff_price = diff_price[diff_price.values<10000]

    # Get the mean
    avg_std = diff_price.median()
    
    return avg_std

def diff_score(df, customer_feature, hotel_feature):
    """
    float [0,1]
    
    customer_feature: str of feature
    hotel_feature: str of feature
    
    
    """
    df_booked = df[df.booking_bool == 1]
    avg_std = per_cent_avg_std(df_booked, customer_feature, hotel_feature)

    score_ = gaussian(df[hotel_feature], df[customer_feature]-df[customer_feature]*avg_std/2 , df[customer_feature]*avg_std)
    
    return avg_std, score_

def diff_score_test(df, customer_feature, hotel_feature, avg_std):
    score_ = gaussian(df[hotel_feature], df[customer_feature]-df[customer_feature]*avg_std/2 , df[customer_feature]*avg_std)
    return score_

In [0]:
# Add new features to the raw data set and create novel target value
def gen_feat(df, target_, stds=None):
    # Parse date_time
    df['date_time'] = pd.to_datetime(df.date_time, format='%Y-%m-%d %H:%M:%S')

    # Time features
    df = get_time_feat(df)

    # Total count of people
    df.insert(24, 'srch_total_count', df['srch_adults_count'] + df['srch_children_count'])

    # Per-srch hotel score
    df = get_hotelworth(df)
    
    # Fill comp_columns with zero
    comp_columns = ['comp1_rate', 'comp1_inv', 'comp1_rate_percent_diff', 'comp2_rate', 'comp2_inv', 'comp2_rate_percent_diff', 'comp3_rate',
     'comp3_inv', 'comp3_rate_percent_diff', 'comp4_rate', 'comp4_inv', 'comp4_rate_percent_diff', 'comp5_rate', 'comp5_inv', 'comp5_rate_percent_diff',
     'comp6_rate', 'comp6_inv', 'comp6_rate_percent_diff', 'comp7_rate', 'comp7_inv', 'comp7_rate_percent_diff', 'comp8_rate', 'comp8_inv', 'comp8_rate_percent_diff']
    for i in comp_columns:
      df[i] = df[i].fillna(0)

    df = df.drop(columns=comp_columns, axis=1, errors="ignore")
    
    if target_:
        df = get_target(df)
        
        # Get rid of all non-clicked searches
        df.insert(2, 'nr_clicks', df.groupby("srch_id")["click_bool"].transform("sum"))
        df = df[df["nr_clicks"] > 0].drop("nr_clicks", axis=1)
        
        # User's preferences
        std_price, df['price_user_score'] = diff_score(df, 'visitor_hist_adr_usd', 'price_usd')
        std_star1, df['star1_user_score'] = diff_score(df, 'visitor_hist_starrating', 'prop_starrating')
        std_star2, df['star2_user_score'] = diff_score(df, 'visitor_hist_starrating', 'prop_review_score')
 
        stds = []
        stds.append(std_price)
        stds.append(std_star1)
        stds.append(std_star2)

        df.price_user_score = df.price_user_score.fillna(0)
        df.star1_user_score = df.star1_user_score.fillna(0)
        df.star2_user_score = df.star2_user_score.fillna(0)

        df['user_score_perf'] = (df.price_user_score + df.star1_user_score + df.star2_user_score) /3
        
        return stds, df

    else:
        df['price_user_score'] = diff_score_test(df, 'visitor_hist_adr_usd', 'price_usd', stds[0])
        df['star1_user_score'] = diff_score_test(df, 'visitor_hist_starrating', 'prop_starrating', stds[1])
        df['star2_user_score'] = diff_score_test(df, 'visitor_hist_starrating', 'prop_review_score', stds[2])
        
        df.price_user_score = df.price_user_score.fillna(0)
        df.star1_user_score = df.star1_user_score.fillna(0)
        df.star2_user_score = df.star2_user_score.fillna(0)

        df['user_score_perf'] = (df.price_user_score + df.star1_user_score + df.star2_user_score) /3
        
        return df

In [0]:
training_set = pd.read_csv('training_set_VU_DM.csv', nrows=round(DATA_FRACTION*train_rows))
stds, df = gen_feat(training_set, True)

In [0]:
df[df.srch_id == 6].hotel_worth.corr(df[df.srch_id == 6].target)

0.7312408144067815

In [0]:
df[df.srch_id == 6].hotel_worth = df_temp.hotel_worth



,srch_id,weekday,month,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,position,price_usd,promotion_flag,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_total_count,srch_saturday_night_bool,srch_query_affinity_score,orig_destination_distance,random_bool,click_bool,gross_bookings_usd,booking_bool,diff_price_per_srch,diff_star1_per_srch,diff_star2_per_srch,diff_loc1_per_srch,diff_loc2_per_srch,hotel_worth,target,price_user_score,star1_user_score,star2_user_score,user_score_perf
60,6,2,6,14,100,NaN,NaN,100,10759,0,2.0,0,1.95,NaN,0.0,4,97.63,0,21106,1,5,2,0,1,2,0,NaN,652.84,0,0,NaN,0,1.166278,0.000000,0.4,0.661017,0.0,0.000000,0.012513,0.0,0.0,0.0,0.0
61,6,2,6,14,100,NaN,NaN,100,22135,0,5.0,0,1.95,NaN,0.0,6,115.03,0,21106,1,5,2,0,1,2,0,NaN,652.84,0,0,NaN,0,0.991382,0.000000,1.0,0.661017,0.0,0.087877,0.007903,0.0,0.0,0.0,0.0
62,6,2,6,14,100,NaN,NaN,100,52376,2,0.0,1,1.95,NaN,0.0,2,86.03,0,21106,1,5,2,0,1,2,0,NaN,652.85,0,0,NaN,0,1.321728,0.666667,0.0,0.661017,0.0,0.660710,0.026001,0.0,0.0,0.0,0.0
63,6,2,6,14,100,NaN,NaN,100,104251,3,4.0,1,1.95,NaN,0.0,1,145.00,0,21106,1,5,2,0,1,2,0,NaN,652.84,0,1,162.38,1,0.787877,1.000000,0.8,0.661017,0.0,1.000000,0.893584,0.0,0.0,0.0,0.0
64,6,2,6,14,100,NaN,NaN,100,118866,2,4.5,1,1.95,NaN,0.0,3,183.66,0,21106,1,5,2,0,1,2,0,NaN,652.78,0,0,NaN,0,0.622929,0.666667,0.9,0.661017,0.0,0.566277,0.017076,0.0,0.0,0.0,0.0


0.7312408144067812

In [0]:

# df[df.srch_id==101].sort_values(['hotel_worth'], ascending=[False])
# # days, window, adults, children

In [0]:
# print(df.diff_price_per_srch.corr(df.target))


In [0]:
# df[df.booking_bool==1].sort_values(['srch_length_of_stay'], ascending=[True]).head()


TypeError: ignored

# Remove outliers

In [0]:
# Find and remove outliers for each pre-defined feature
def remove_outliers(input_data):
    # Focus on these features, because IDs and Booleans do not make sense for outlier detection
    features = ['prop_starrating', 'prop_location_score1', 
                'prop_log_historical_price', 'price_usd', 'srch_length_of_stay', 'srch_booking_window', 
                'srch_adults_count', 'srch_children_count', 'srch_room_count']
        
    indices_to_remove = []
  
    # Build the boundaries first, then find and remove outliers or replace them with their boundary
    for column in features:
        if is_numeric_dtype(input_data[column]):

            # Calculate the interquartile range
            Q1 = input_data[column].quantile(0.25)
            Q3 = input_data[column].quantile(0.75)
            IQR = Q3 - Q1
            
            if IQR == 0:
                continue

            # Calculate the maximum value and minimum values according to the Tukey rule
            max_value = Q3 + 1.5 * IQR
            min_value = Q1 - 1.5 * IQR
            
            # Iteratively build a list of indices to be removed
            # Do not remove in place, because that will change the DataFrame and influence subsequent interquartile ranges for other columns
            indices_to_remove = indices_to_remove + list(input_data.loc[(input_data[column].notnull()) & ((input_data[column] < min_value) | (input_data[column] > max_value))].index)

    #for index in indices_to_remove:
    indices_to_remove = list(set(indices_to_remove))
    output_data = pd.concat([input_data[input_data["target"] >= 0.4], input_data[input_data["target"] < 0.4].drop(index=indices_to_remove, errors="ignore")], axis=0)
    return output_data

# Handle missing data

In [0]:
# Drop all columns that have at least one NaN value
def remove_nans(df):
    # For speed, we drop NaN values
    columns_with_na = df.columns[df.isnull().any()]
    df = df.drop(columns_with_na, axis=1)
    
    return df

# Replace NaN and Inf values of all numeric columns
def replace_nans(data, unique_nan_values):
  
    # features = get_features(with_target=False)
    
    data = data.replace([np.inf, -np.inf], np.nan)
    
    for column in list(data):

        if is_numeric_dtype(data[column]):
            data[column] = data[column].fillna(unique_nan_values[column])
    
    return data

# Balance the data

In [0]:
# Removes non-clicked rows randomly up to a final ratio of 50/50 for clicked to non-clicked
def balance_data(df, print_=False):
    click_indices = df[df.click_bool == 1].index
    random_indices = np.random.choice(click_indices, len(df.loc[df.click_bool == 1]), replace=False)
    click_sample = df.loc[random_indices]

    not_click = df[df.click_bool == 0].index
    random_indices = np.random.choice(not_click, sum(df['click_bool']), replace=False)
    not_click_sample = df.loc[random_indices]

    df_new = pd.concat([not_click_sample, click_sample], axis=0)

    if print_:
        print("Percentage of not click impressions: ", len(df_new[df_new.click_bool == 0])/len(df_new))
        print("Percentage of click impression: ", len(df_new[df_new.click_bool == 1])/len(df_new))
        print("Total number of records in resampled data: ", len(df_new))

    df = df_new.sort_values(['srch_id'], ascending=[True])
    df = df.reset_index(drop=True)
    
    return df

# Split data

In [0]:
# Splits the data into a training and validation set
def split_data(train, ratio=0.7):
    
    temp_train = copy.deepcopy(train)
    
    uniq_searches = temp_train.srch_id.unique().tolist()
    train_id = random.sample(uniq_searches, round(len(uniq_searches)*ratio))
    
    val_id = list(set(uniq_searches) - set(train_id))
    
    train_set = temp_train[temp_train['srch_id'].isin(train_id)]
    validation_set = temp_train[temp_train['srch_id'].isin(val_id)]
    
    return train_set, validation_set

# Splits the data into features and targets
def prepare_features_and_targets(raw_df, testing=False):
    list_of_columns_to_exclude = ['position', 'click_bool', 'booking_bool', 'target']

    if testing:
        return raw_df.drop(list_of_columns_to_exclude, axis=1, errors='ignore')
    
    df_t = raw_df.target
    df_x = raw_df.drop(list_of_columns_to_exclude, axis=1, errors='ignore')
    
    return df_x, df_t

# Model

In [0]:
# Trains a model on the given input and returns the predictions on the given validation set
def regressor_fit(x_train, t_train, x_val, eval_on_train=False):
    # Init model

#     model = GradientBoostingRegressor(learning_rate=0.1, n_estimators=100,
#                                      min_samples_split=2, min_samples_leaf=1,
#                                      max_depth=2)

#     model = AdaBoostRegressor(random_state=0, learning_rate=0.1, loss='square' ,n_estimators=65)
    model = RandomForestRegressor(n_estimators=51,min_samples_leaf=5,min_samples_split=3)


    # fit
    model.fit(x_train, t_train)
    if eval_on_train:
        y_train = model.predict(x_train)
        errors = abs(y_train - t_train)
        print('Mean Absolute Error on Train:', round(np.mean(errors), 2), 'degrees.')
    
    y_val = model.predict(x_val)
    return y_val

# Ranking

In [0]:
# Returns a dataframe that has been sorted with respect to predictions per srch_id
def rank_df(data_to_be_ranked, predictions, validation=True):    
    
    df = data_to_be_ranked[['srch_id', 'prop_id']]
    df.insert(2, 'predictions', predictions)
    
    if validation:
        df.insert(3, 'target', data_to_be_ranked['target'])
        df.insert(4, 'relevance', 4 * data_to_be_ranked['booking_bool'] + data_to_be_ranked['click_bool'])

    # Sort them to get a ranking
    df = df.sort_values(['srch_id', 'predictions'], ascending=[True, False])
    
    if not validation:
        # Set file name
        time = str(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))
        file_name = 'submission_' + time + '.csv'

        df = df.drop(['predictions'],axis=1)
        df.to_csv(file_name, header=True, index=False)
    return df

In [0]:
def compute_dcg(ranking, cut=5):
    len_ = min(cut, len(ranking))
    dcg = ranking[:len_]    
    result = 0
    for i in range(1, len_):
        result += (dcg[i-1]) / math.log(i+1, 2)
    return result

def compute_ndcg(val_set, val_pred):
    ranked_val_set = rank_df(val_set, val_pred, validation=True)
    ndcgs = []
    for idx in ranked_val_set.srch_id.unique():

        our_ranking = ranked_val_set[ranked_val_set['srch_id'] == idx]["relevance"].values
        our_dcg = compute_dcg(our_ranking)

        best_ranking = sorted(our_ranking, reverse=True)
        best_dcg = compute_dcg(best_ranking)
        
        if best_dcg > 0:
            ndcg = our_dcg / best_dcg
        else:
            ndcg = 0

        ndcgs.append(ndcg)
    return sum(ndcgs) / len(ndcgs)

# Code execution

In [0]:
# Preprocessing
stds, df = gen_feat(training_set, True)
df = remove_nans(df)
if TESTING:
    df_test = gen_feat(test_set, False, stds)
    df_test = remove_nans(df_test)

# Two cases: Validation or Testing!
if not TESTING:
    ndcg_cross_validation = []
    folds = 3
    
    for i in range(folds):
        
        # Prepare training and validation for model
        train_set, val_set = split_data(df, 0.95)
        
        # Potential additional preprocessing steps
        
        #train_set = remove_outliers(train_set)
        train_set = balance_data(train_set)
        
        # Use this print statement to figure out a good split ratio
        print("Shapes", train_set.shape, val_set.shape)
        
        train_x, train_t = prepare_features_and_targets(train_set)
        val_x, val_t = prepare_features_and_targets(val_set)

        # Predict and rank validation set
        val_y = regressor_fit(train_x, train_t, val_x)
        ranking = rank_df(val_set, val_y, validation=True)
        ndcg = compute_ndcg(val_set, val_y)
        ndcg_cross_validation.append(ndcg)

    print("All NDCGs", ndcg_cross_validation)
    print("Mean:", sum(ndcg_cross_validation) / len(ndcg_cross_validation))

else:

    # Prepare training and validation for model
    train_set = balance_data(df)
    train_x, train_t = prepare_features_and_targets(train_set)
    test_x = prepare_features_and_targets(df_test, testing=True)

    # Predict and rank validation set
    test_y = regressor_fit(train_x, train_t, test_x)
    ranking = rank_df(test_x, test_y, validation=False)

Shapes (84372, 36) (49196, 36)
Shapes (84320, 36) (48448, 36)
Shapes (84320, 36) (48641, 36)
All NDCGs [0.3059036927754511, 0.3218183364092447, 0.29969679711668257]
Mean: 0.3091396087671261


# Exploration
